# Version 2 Workspace

In [ ]:
!pip install --upgrade py3Dmol accelerate

In [1]:
from transformers import AutoTokenizer, EsmForProteinFolding

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)

model = model.cuda()

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import torch

torch.backends.cuda.matmul.allow_tf32 = True

In [3]:
test_protein = "MGAGASAEEKHSRELEKKLKEDAEKDARTVKLLLLGAGESGKSTIVKQMKIIHQDGYSLEECLEFIAIIYGNTLQSILAIVRAMTTLNIQYGDSARQDDARKLMHMADTIEEGTMPKEMSDIIQRLWKDSGIQACFERASEYQLNDSAGYYLSDLERLVTPGYVPTEQDVLRSRVKTTGIIETQFSFKDLNFRMFDVGGQRSERKKWIHCFEGVTCIIFIAALSAYDMVLVEDDEVNRMHESLHLFNSICNHRYFATTSIVLFLNKKDVFFEKIKKAHLSICFPDYDGPNTYEDAGNYIKVQFLELNMRRDVKEIYSHMTCATDTQNVKFVFDAVTDIIIKENLKDCGLF"

tokenized_input = tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)['input_ids']
tokenized_input = tokenized_input.cuda()

with torch.no_grad():
    output = model(tokenized_input)


# Real Structual Embedding Generations

## Load Dataset

In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [2]:
from proteinclip.fasta_utils import read_fasta
fasta_data_raw = read_fasta("./data/uniprot/uniprot_sprot.fasta.gz")
fasta_data_raw[list(fasta_data_raw.keys())[0]]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
INFO:root:Read 572970 sequences from ./data/uniprot/uniprot_sprot.fasta.gz


'MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGHFSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTKQHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDFKALVESAHRMRQGHMINVKYILYQLLKKHGHGPDGPDILTVKTGSKGVLYDDSFRKIYTDLGWKFTPL'

In [3]:
# filter out excessive length
# officially doesn't support sequences with > 1024 residuals

fasta_data = {k: v for k, v in fasta_data_raw.items() if len(v) <= 1024}
print(f"Eliminating: {100*(1 - len(fasta_data) / len(fasta_data_raw)) : .2f}% of dataset for excessive length")

Eliminating:  3.17% of dataset for excessive length


In [4]:
protein_ids = list(fasta_data.keys())
sequences = list(fasta_data.values())

In [5]:
from transformers import AutoTokenizer, EsmForProteinFolding

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)

model = model.cuda()

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
print(torch.cuda.memory_summary())


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  16690 MiB |  16690 MiB |  16690 MiB |    512 B   |
|       from large pool |  16686 MiB |  16686 MiB |  16686 MiB |      0 B   |
|       from small pool |      4 MiB |      4 MiB |      4 MiB |    512 B   |
|---------------------------------------------------------------------------|
| Active memory         |  16690 MiB |  16690 MiB |  16690 MiB |    512 B   |
|       from large pool |  16686 MiB |  16686 MiB |  16686 MiB |

In [6]:

from tqdm.auto import tqdm 
# Define a batch size (adjust based on your available resources)
batch_size = 10

# Initialize list to store embeddings
embeddings_dict = {"protein_id": [], "embedding": []}

for i in tqdm(range(0, len(sequences), batch_size), total=(len(sequences) // batch_size) + 1):
    batch_protein_ids = protein_ids[i:i+batch_size]
    batch_sequences = sequences[i:i+batch_size]

    tokenized_input = tokenizer(batch_sequences, return_tensors="pt", add_special_tokens=False, truncation=True, max_length=1024, padding=True)['input_ids']
    tokenized_input = tokenized_input.cuda()

    with torch.no_grad():
        output = model(tokenized_input)


    # Convert the batch embeddings to list and pair with corresponding protein IDs
    embedding_vectors = output.tolist()
    embeddings_dict["protein_id"].extend(protein_ids)
    embeddings_dict["embedding"].extend(embedding_vectors)

embedding_df = pd.DataFrame(embeddings)

# Save the DataFrame as a Parquet file
embedding_df.to_parquet("data/structural/esmfold_embeddings.parquet", index=False)
print("Embeddings saved to data/structural/esmfold_embeddings.parquet")


  0%|          | 0/55484 [00:00<?, ?it/s]

AttributeError: 'EsmForProteinFoldingOutput' object has no attribute 'tolist'

In [13]:
output.unnormalized_angles.shape

torch.Size([8, 10, 458, 7, 2])

In [12]:
output?

Type:        EsmForProteinFoldingOutput
String form:
EsmForProteinFoldingOutput(frames=tensor([[[[ 1.6310e-01, -7.6539e-01,  4.3753e-01,  ..., -1.7970 <...> 0.2513]]],
           device='cuda:0'), max_predicted_aligned_error=tensor(31.7500, device='cuda:0'))
Length:      23
File:        ~/.conda/envs/compbio/lib/python3.13/site-packages/transformers/models/esm/modeling_esmfold.py
Docstring:  
Output type of [`EsmForProteinFoldingOutput`].

Args:
    frames (`torch.FloatTensor`):
        Output frames.
    sidechain_frames (`torch.FloatTensor`):
        Output sidechain frames.
    unnormalized_angles (`torch.FloatTensor`):
        Predicted unnormalized backbone and side chain torsion angles.
    angles (`torch.FloatTensor`):
        Predicted backbone and side chain torsion angles.
    positions (`torch.FloatTensor`):
        Predicted positions of the backbone and side chain atoms.
    states (`torch.FloatTensor`):
        Hidden states from the protein folding trunk.
    s_s (`torch

In [6]:
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37

def convert_outputs_to_pdb(outputs):
    final_atom_positions = atom14_to_atom37(outputs["positions"][-1], outputs)
    outputs = {k: v.to("cpu").numpy() for k, v in outputs.items()}
    final_atom_positions = final_atom_positions.cpu().numpy()
    final_atom_mask = outputs["atom37_atom_exists"]
    pdbs = []
    for i in range(outputs["aatype"].shape[0]):
        aa = outputs["aatype"][i]
        pred_pos = final_atom_positions[i]
        mask = final_atom_mask[i]
        resid = outputs["residue_index"][i] + 1
        pred = OFProtein(
            aatype=aa,
            atom_positions=pred_pos,
            atom_mask=mask,
            residue_index=resid,
            b_factors=outputs["plddt"][i],
            chain_index=outputs["chain_index"][i] if "chain_index" in outputs else None,
        )
        pdbs.append(to_pdb(pred))
    return pdbs